In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from publicdata.census.files.metafiles import TableLookup
import shelve 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()

# Setup importing form this package
pkg.set_sys_path()
import pylib # Import the code for the package
from pylib.census_path_processing import * 

pkg

# Metadata for the 2020 ACS
`civicknowledge.com-census_meta-2020e5-2.1.1` Last Update: 2023-01-13T01:02:55

_Deatilsed metadata for ACS releases._




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [paths](notebooks/build.ipynb#paths_df)**. Column paths, processed by GPT
* ** [metadata](notebooks/build.ipynb#mdf)**. Combined columns and tables
* ** [tables](notebooks/build.ipynb#tdf)**. 
* ** [columns](notebooks/build.ipynb#cdf)**.

In [3]:
year = 2020
release = 5

tl = TableLookup(year, release)

In [4]:
tdf = tl.tables_df
cdf = tl.columns_df

cdf = cdf[['uid', 'table_id', 'name', 'sex', 'raceeth', 'age', 'poverty_status', 'filtered_pathname']]\
    .rename(columns={'filtered_pathname':'path', 'uid':'column_id'})

tdf = tdf[['table_id', 'title', 'bare_title', 'stripped_title','universe', 'subject', 'race',
       'age', 'sex']].copy().rename(columns={'race':'raceeth'})

tdf.head()

,table_id,title,bare_title,stripped_title,universe,subject,raceeth,age,sex
0,B01001,Sex By Age,Sex By Age,Population,Total population,Age-Sex,all,all,all
1,B01001A,Sex By Age (White Alone),Sex By Age,Population,People who are White alone,Age-Sex,white,all,all
2,B01001B,Sex By Age (Black Or African American Alone),Sex By Age,Population,Black or African American alone,Age-Sex,black,all,all
3,B01001C,Sex By Age (American Indian And Alaska Native ...,Sex By Age,Population,People who are American Indian and Alaska Nati...,Age-Sex,aian,all,all
4,B01001D,Sex By Age (Asian Alone),Sex By Age,Population,People who are Asian alone,Age-Sex,asian,all,all


In [5]:
cdf.head()

,column_id,table_id,name,sex,raceeth,age,poverty_status,path
0,B01001_001,B01001,Total,all,all,all,all,
1,B01001_002,B01001,Male,male,all,all,all,
2,B01001_003,B01001,Under 5 years,male,all,000-005,all,
3,B01001_004,B01001,5 to 9 years,male,all,005-009,all,
4,B01001_005,B01001,10 to 14 years,male,all,010-014,all,


In [6]:

trename_map = { k:'table_'+k for k in 'raceeth age sex'.split()}
#crename_map = { k:'col_'+k for k in 'raceeth age sex'.split()}
mdf=tdf.rename(columns=trename_map).merge(cdf, on='table_id').copy()

def mk_update_f(table_col_name, col_name):

    def _update(r):
        """ If the column isn't restricted, but the table is, then the column should be marked as restricted."""
        if r[table_col_name] != 'all' and r[col_name] == 'all':
            r[col_name] = r[table_col_name]

        return r
    
    return _update

_sex = mk_update_f('table_sex','sex')
_age = mk_update_f('table_age','age')
_raceeth = mk_update_f('table_raceeth','raceeth')

mdf = pd.DataFrame([_raceeth(_age(_sex(r)))  for idx, r in mdf.iterrows()])


mdf[  (mdf.table_sex!='all') ].head().T

,14635,14636,14637,14638,14639
table_id,B23003,B23003,B23003,B23003,B23003
title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
bare_title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
stripped_title,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...,Presence Of Own Children Under 18 Years By Age...
universe,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households,Females 20 to 64 years in households
subject,Employment Status,Employment Status,Employment Status,Employment Status,Employment Status
table_raceeth,all,all,all,all,all
table_age,020-064,020-064,020-064,020-064,020-064
table_sex,female,female,female,female,female
column_id,B23003_001,B23003_002,B23003_003,B23003_004,B23003_005


In [7]:
#
# Clean the pathnames
# 


if False:
    consecutive_err = 0
    while True:
        consecutive_err +=1
        try:
            run_path_cleaning()
            consecutive_err = 0
        except StopIteration:
            print("Done")
            break
        except Exception as e:
            print(e)

        if consecutive_err > 3:
            print("Too many errors")
            break


if False:
    # Turn the database into a dataset, so we can save it in the package, for caching in future builds. 
    # This is not currently loaded yet. 
    with shelve.open('census_paths') as db:
        paths_df = pd.DataFrame(db.values())

    paths_df.head()

if False:
  
    with shelve.open('census_paths') as db:
        print(len(db))
        pdf = pd.DataFrame(db.values()).rename(
                columns={
                    "unique_id": "table_id",
                    "path": "filtered_path",
                    "name": "path_name",
                }
            )


        mdf = add_rest_str(tdf, cdf)

        #mdf = mdf.merge(pdf, on=["table_id", "filtered_path"], how="left").copy()

        mdf["rest_description"] = mdf.apply(make_restricted_description, axis=1)

        mdf["col_desc"] = mdf.stripped_title + " for " + mdf.rest_description

        metadata_df = mdf.rename(columns={'uid':'column_id'})

    metadata_df.head()  